# Statistics on missing CA and CB atoms AND side chain angles

## Aims of this notebook

### 1. Missing CA and CB atoms

In our fingerprint, both the exposure and side chain angle features are dependent on CA and CB atoms.
Here, we investigate where and overall how often these atoms are missing in the KLIFS data.

1. Get for each KLIFS molecule CA and CB atom coordinates per residue position.
2. Calculate missing atom rate per residue position: CA, CB and CA+CB missing.

### 2. Side chain angle (SCA) distribution

Side chain angles describe the angle between Ca, Cb, and residue centroid (without backbone atoms and hydrogens). 

Small amino acids (with tiny side chains) should not show much angle diversion (with smaller angles), larger ones should (with larger angles).

1. Calculate for each amino acid the angle distribution.
2. Save molecule and residue code for each angle, in order to trace back interesting angles.
3. Check diversity of angles per amino acid. If no diversity observed, side chain angle might not be such a good measure, since it does not depend on structural conformation but solely on amino acid type.

## Imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from pathlib import Path
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('../..')
from kinsim_structure.auxiliary import KlifsMoleculeLoader
from kinsim_structure.analysis import GapRate, SideChainAngleGenerator, SideChainAngleAnalyser

sns.set()
%matplotlib inline

In [4]:
warnings.filterwarnings(action='once')

## IO paths

In [5]:
path_to_kinsim = Path('.') / '..' / '..'
path_to_data = path_to_kinsim / 'examples' / 'data'
path_to_results = path_to_kinsim / 'examples' / 'results' / 'features' / 'sca_centroid_wo_backbone' 

## Load KLIFS metadata

In [6]:
klifs_metadata = pd.read_csv(path_to_data / 'postprocessed' / 'klifs_metadata_postprocessed.csv' , index_col=0)

In [7]:
klifs_metadata.shape

(3878, 23)

## Data generation

### Gap rate

In [ ]:
gap_rate = GapRate(klifs_metadata)

### Side chain angle

## Data analysis

### Gap rate

In [ ]:
gap_rate.plot_gap_rate(
    path_to_results
)

### Missing CA and CB atoms

In [ ]:
side_chain_angle_analyser = SideChainAngleAnalyser()
side_chain_angle_analyser.load_data(path_to_results / 'side_chain_angles.csv')

In [ ]:
side_chain_angle_analyser.data.head()

In [ ]:
side_chain_angle_analyser.data.shape

In [ ]:
side_chain_angle_analyser.get_missing_residues_ca_cb(gap_rate)

In [ ]:
side_chain_angle_analyser.plot_missing_residues_ca_cb(
    path_to_results
)

In [ ]:
# How many residues have a missing Cb but are not GLY?
side_chain_angle_analyser.data[
    (side_chain_angle_analyser.data.cb.isna()) &
    (side_chain_angle_analyser.data.residue_name != 'GLY')
].shape

### SCA angle distribution

In [ ]:
side_chain_angle_analyser.plot_side_chain_angle_distribution(
    path_to_results, 
    kind='violin'
)

In [ ]:
side_chain_angle_analyser.plot_side_chain_angle_distribution(
    path_to_results, 
    kind='histograms'
)

### SCA statistics

In [ ]:
side_chain_angle_analyser.data[['residue_name', 'sca']].groupby('residue_name').describe()

Look at extreme (unexpected values)...

In [ ]:
side_chain_angle_analyser.data[
    (side_chain_angle_analyser.data.residue_name == 'TYR') & (side_chain_angle_analyser.data.sca == 180)
].head()

### Visualize SCAs

In [8]:
import time

import nglview as nv

from kinsim_structure.auxiliary import split_klifs_code

_ColormakerRegistry()

/home/dominique/anaconda2/envs/kinsim/lib/python3.6/site-packages/nglview/widget.py:162: DeprecationWarning: Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.
  _ngl_view_id = List(Unicode).tag(sync=True)


In [ ]:
sca_data_grouped = list(side_chain_angle_analyser.data.groupby('klifs_code'))
sca_data = sca_data_grouped[1][1]

In [ ]:
def create_viewer(sca_data):

    klifs_code = split_klifs_code(sca_data.klifs_code.iloc[0])

    pdb_id = klifs_code['pdb_id']
    chain = klifs_code['chain']

    viewer = nv.NGLWidget()
    viewer.add_pdbid(pdb_id)
    viewer.add_representation(repr_type='cartoon', selection='all')
    viewer.remove_ball_and_stick()

    return viewer

In [ ]:
def show_sca_features(viewer, sca_data): 

    klifs_code = split_klifs_code(sca_data.klifs_code.iloc[0])

    pdb_id = klifs_code['pdb_id']
    chain = klifs_code['chain']
    
    # Representation parameters
    sphere_radius = 0.3

    colors = {
        'ca': [1, 0, 0],
        'cb': [0, 1, 0],
        'centroid': [0, 0, 1]
    }

    # Show side chain angle feature per residue
    for index, row in sca_data.iterrows():

        res_id = row.residue_id

        try:
            ca = list(row.ca.get_array())
        except AttributeError:
            pass

        try:
            cb = list(row.cb.get_array())
        except AttributeError:
            pass

        try:
            centroid = list(row.centroid.get_array())
        except AttributeError:
            pass

        selection = f'{res_id}:{chain}'

        viewer.add_representation(repr_type='line', selection=selection)
        viewer.shape.add_sphere(ca, colors['ca'], sphere_radius)
        viewer.shape.add_sphere(cb, colors['cb'], sphere_radius)
        viewer.shape.add_sphere(centroid, colors['centroid'], sphere_radius)

In [ ]:
def center_on(viewer, sca_data, res_id):
    
    klifs_code = split_klifs_code(sca_data.klifs_code.iloc[0])

    pdb_id = klifs_code['pdb_id']
    chain = klifs_code['chain']
    
    selection = f'{res_id}:{chain}'
        
    viewer.remove_ball_and_stick()
    viewer.add_ball_and_stick(selection=selection)
    viewer.center(selection=selection)

In [ ]:
viewer = create_viewer(sca_data)
viewer

In [ ]:
show_sca_features(viewer, sca_data)

In [ ]:
center_on(viewer, sca_data, 1121)

### SCA mean and median

Get mean and median of side chain angles per amino acid and save to file. 
Use these values for residues with missing Ca/Cb atoms.

In [ ]:
side_chain_angle_analyser.get_mean_median(
    from_file=path_to_results / 'stats_missing_ca_cb_and_sca.p'
)